In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import requests

In [2]:
url = 'https://www.goodreads.com/shelf/show/fiction'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

In [3]:
book_title = [item.text for item in soup.find_all(attrs={'class':'bookTitle'})]
book_title

['To Kill a Mockingbird (To Kill a Mockingbird, #1)',
 '1984 (Kindle Edition)',
 'The Great Gatsby (Paperback)',
 "Harry Potter and the Sorcerer's Stone (Harry Potter, #1)",
 'Animal Farm (Mass Market Paperback)',
 'The Catcher in the Rye (Paperback)',
 'The Hunger Games (The Hunger Games, #1)',
 'The Da Vinci Code (Robert Langdon, #2)',
 'The Kite Runner (Paperback)',
 "The Handmaid's Tale (Paperback)",
 'Lord of the Flies (Paperback)',
 'Pride and Prejudice (Paperback)',
 'Harry Potter and the Chamber of Secrets (Harry Potter, #2)',
 'The Hobbit or There and Back Again (Paperback)',
 'Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)',
 'The Alchemist (Paperback)',
 'Harry Potter and the Goblet of Fire (Harry Potter, #4)',
 'Life of Pi (Paperback)',
 'Catching Fire (The Hunger Games, #2)',
 'Fahrenheit 451 (Kindle Edition)',
 'Harry Potter and the Deathly Hallows (Harry Potter, #7)',
 'The Girl with the Dragon Tattoo (Millennium, #1)',
 'Harry Potter and the Half-Blood Prin

In [4]:
soup.find_all(attrs={'class':'greyText smallText'})[0].text.split('—\n')

['\n                avg rating 4.27 ',
 '                3,992,272 ratings  ',
 '                published 1960\n              ']

In [5]:
year = [int(re.findall("published (\d{4})", item.text.split('—\n')[2])[0]) for item in soup.find_all(attrs={'class':'greyText smallText'})]
year

[1960,
 1949,
 1925,
 1997,
 1945,
 1951,
 2008,
 2003,
 2003,
 1985,
 1954,
 1813,
 1998,
 1937,
 1999,
 1988,
 2000,
 2001,
 2009,
 1953,
 2007,
 2005,
 2005,
 2003,
 1932,
 2010,
 2005,
 2003,
 2009,
 1937,
 2012,
 2000,
 1969,
 1847,
 2012,
 2003,
 2002,
 1996,
 1997,
 2006,
 2006,
 2015,
 1979,
 1961,
 1884,
 1967,
 1954,
 1818,
 1890,
 2014]

In [6]:
avg_rating = [float(re.findall("\d+\.\d+", item.text.split('—\n')[0])[0]) for item in soup.find_all(attrs={'class':'greyText smallText'})]
avg_rating

[4.27,
 4.17,
 3.91,
 4.47,
 3.92,
 3.8,
 4.33,
 3.83,
 4.28,
 4.09,
 3.67,
 4.25,
 4.41,
 4.27,
 4.56,
 3.85,
 4.55,
 3.9,
 4.29,
 3.98,
 4.62,
 4.13,
 4.56,
 4.49,
 3.98,
 4.03,
 4.37,
 3.87,
 4.46,
 3.86,
 4.06,
 3.89,
 4.07,
 4.11,
 4.23,
 3.96,
 3.8,
 4.45,
 4.1,
 4.08,
 3.96,
 3.91,
 4.22,
 3.98,
 3.81,
 4.06,
 4.35,
 3.78,
 4.07,
 4.33]

In [7]:
total_ratings = [int(re.findall("(\d{1,3}(,\d{3})*)", item.text.split('—\n')[1])[0][0].replace(',', '')) for item in soup.find_all(attrs={'class':'greyText smallText'})]
total_ratings

[3992272,
 2651064,
 3333413,
 6056179,
 2388103,
 2458586,
 5781181,
 1739583,
 2189730,
 1150996,
 2014205,
 2617187,
 2341319,
 2561818,
 2386572,
 1790269,
 2233711,
 1208710,
 2220292,
 1444421,
 2391631,
 2320962,
 2129575,
 2182805,
 1280489,
 2089758,
 1602972,
 1073162,
 1904266,
 1748111,
 2013241,
 2411610,
 1029265,
 1460506,
 3083651,
 1457499,
 1852718,
 1788642,
 1584624,
 1260163,
 647729,
 1755341,
 1251081,
 668386,
 1086247,
 669767,
 2125711,
 1035572,
 833812,
 828546]

In [8]:
books = pd.DataFrame(
    zip(book_title, avg_rating, total_ratings, year), 
    columns=['Title', 'Average ratings', 'Total ratings', 'Year published']
)
books

,Title,Average ratings,Total ratings,Year published
0,"To Kill a Mockingbird (To Kill a Mockingbird, #1)",4.27,3992272,1960
1,1984 (Kindle Edition),4.17,2651064,1949
2,The Great Gatsby (Paperback),3.91,3333413,1925
3,Harry Potter and the Sorcerer's Stone (Harry P...,4.47,6056179,1997
4,Animal Farm (Mass Market Paperback),3.92,2388103,1945
5,The Catcher in the Rye (Paperback),3.80,2458586,1951
6,"The Hunger Games (The Hunger Games, #1)",4.33,5781181,2008
7,"The Da Vinci Code (Robert Langdon, #2)",3.83,1739583,2003
8,The Kite Runner (Paperback),4.28,2189730,2003
9,The Handmaid's Tale (Paperback),4.09,1150996,1985


In [9]:
books['Year published rounded'] = books['Year published'].round(decimals=-1)
books

,Title,Average ratings,Total ratings,Year published,Year published rounded
0,"To Kill a Mockingbird (To Kill a Mockingbird, #1)",4.27,3992272,1960,1960
1,1984 (Kindle Edition),4.17,2651064,1949,1950
2,The Great Gatsby (Paperback),3.91,3333413,1925,1920
3,Harry Potter and the Sorcerer's Stone (Harry P...,4.47,6056179,1997,2000
4,Animal Farm (Mass Market Paperback),3.92,2388103,1945,1940
5,The Catcher in the Rye (Paperback),3.80,2458586,1951,1950
6,"The Hunger Games (The Hunger Games, #1)",4.33,5781181,2008,2010
7,"The Da Vinci Code (Robert Langdon, #2)",3.83,1739583,2003,2000
8,The Kite Runner (Paperback),4.28,2189730,2003,2000
9,The Handmaid's Tale (Paperback),4.09,1150996,1985,1980


In [10]:
books.groupby('Year published rounded').size()

Year published rounded
1810     1
1820     1
1850     1
1880     1
1890     1
1920     1
1930     1
1940     3
1950     5
1960     2
1970     2
1980     2
1990     1
2000    17
2010    10
2020     1
dtype: int64

# Extrakce tabulky

Lze použit funkci `read_html`, která vyžaduje HTML řetězec (objekt `BeautifulSoup` musí být převeden na `str`).

In [11]:
url = 'https://www.czso.cz/csu/czso/aktualniinformace'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

In [12]:
economic_stat = pd.read_html(str(soup.find(class_="table nu-table")), header=0)[0]
economic_stat

,Ukazatel,Období,Meziroční růst/pokles (v %),Datum zveřejnění
0,Hrubý domácí produkt,2. čtvrtletí 2019,27,14.08.2019
1,Index spotřebitelských cen,červenec 2019,29,12.08.2019
2,Míra inflace,červenec 2019,26,12.08.2019
3,Průmyslová produkce,červen 2019,-38,06.08.2019
4,Stavební produkce,červen 2019,24,06.08.2019
5,Tržby v maloobchodě (CZ-NACE 47),červen 2019,46,07.08.2019
6,Průměrná mzda - nominální,1. čtvrtletí 2019,74,04.06.2019
7,Průměrná mzda - reálná,1. čtvrtletí 2019,46,04.06.2019
8,Indexy cen výrobců - zemědělských,červen 2019,129,17.07.2019
9,Indexy cen výrobců - průmyslových,červen 2019,25,17.07.2019


# Cvičení 6.2

Ze stránek [ministersva financí](https://www.mfcr.cz/cs/verejny-sektor/statni-rozpocet/legislativa-statniho-rozpoctu/2019/rozpocet-resortu-financi-kapitola-312-20-33908) vyextrahujte tabulku rozpočtu.